In [1]:
import pandas as pd
import spacy
import ast
import re
from pprint import pprint
import json
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from sklearn import linear_model
from sklearn.feature_extraction.text import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, r2_score, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from scipy import sparse
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yubozhang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
reviews = []
with open('scraped_reviews_slack.txt', 'r') as f: #, encoding='utf-8'
    reviews = f.read()

reviews = re.sub('\]', '', reviews)
reviews = re.sub('\[', '', reviews)
reviews = re.sub('\n', '', reviews)
reviews = re.sub("'", '"', reviews)
reviews = re.sub('    ', ' ', reviews)
reviews = re.sub(' {', '{', reviews)
reviews = re.sub(' }', '}', reviews)
reviews = re.sub('{ ', '{', reviews)
reviews = re.sub('} ', '}', reviews)
reviews = re.sub('}  {', '}, {', reviews)
reviews = re.sub('}{', '}, {', reviews)
reviews = re.sub('id:', '"id":', reviews)
reviews = re.sub('userName:', '"userName":', reviews)
reviews = re.sub('userImage:', '"userImage":', reviews)
reviews = re.sub('date:', '"date":', reviews)
reviews = re.sub('url:', '"url":', reviews)
reviews = re.sub('score:', '"score":', reviews)
reviews = re.sub('title:', '"title":', reviews)
reviews = re.sub('text:', '"text":', reviews)
reviews = re.sub('replyDate:', '"replyDate":', reviews)
reviews = re.sub('replyText:', '"replyText":', reviews)
reviews = re.sub(': undefined', ': "undefined"', reviews)

In [3]:
review_strings = [r+'}' for r in reviews.split('},')]
review_strings[-1] = review_strings[-1][:-1]

In [4]:
review_list = []
for r in review_strings:
    try:
        review_list.append(json.loads(r))
    except:
        pass

In [5]:
raw_corpus = []
for i in range(1000): # change to larger number
    raw_corpus.append(review_list[i][u'text'])

In [6]:
data = raw_corpus

In [7]:
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]
data = [re.sub("\"", "", sent) for sent in data]
data = [re.sub("_", "", sent) for sent in data]

data = [re.sub("good'", "", sent) for sent in data]
data = [re.sub("app", "", sent) for sent in data]
data = [re.sub("nice", "", sent) for sent in data]
pprint(data[1:10])


[u'Its not well designed for phones. Its hard to navigate between many conversations and its super buggy. I see notification dots but there are no unread messages.',
 u'The by far most annoying and impossible to configure ',
 u'Helped my grad student organize her thesis project',
 u'Payter!',
 u'Thanks for your slack',
 u'Storage space is low. Tanks',
 u'Love slack on Mac and iOS. First time using on Android. Good so far.',
 u'Edited. This is now fixed. I think it was a change on the windows  maybe but this just started working correctly. The team also put specific content on Android settings that also helped but i am now getting notifications and my dev team likes me (a bit) more! Old review: Need help with Oreo and Notifications. I dont think your Android  team gets how to keep the  sending notifications in Oreo given features to reduce battery consumption. My dev team hates that I dont get their notifications if I am not at my desk, but Slack seems to always be shut down on my Onepl

In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str([sentence]), deacc=True)) 


In [10]:
data_words = list(sent_to_words(data))

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
#print(trigram_mod[bigram_mod[data_words[0][:30]]])
#print(list(sent_to_words(data)))

In [11]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        if sent != []:
            doc = nlp(" ".join(sent)) 

            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
 
    return texts_out

In [12]:
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized)

[[u'well', u'design', u'phone', u'hard', u'navigate', u'many', u'conversation', u'super', u'buggy', u'see', u'notification', u'dot', u'unread', u'message'], [u'far', u'annoying', u'impossible', u'configure'], [u'help', u'grad', u'student', u'organize', u'thesis', u'project'], [u'payter'], [u'thank', u'slack'], [u'storage', u'space', u'low', u'tank'], [u'love', u'slack', u'mac', u'ios', u'first', u'time', u'use', u'android', u'good', u'far'], [u'edit', u'fix', u'think', u'change', u'window', u'maybe', u'start', u'work', u'correctly', u'team', u'also', u'put', u'specific', u'content', u'android', u'setting', u'also', u'help', u'get', u'notification', u'dev', u'team', u'like', u'bit', u'old', u'review', u'need', u'help', u'oreo', u'notification', u'do', u'not', u'think', u'android', u'team', u'get', u'keep', u'send', u'notification', u'oreo', u'give', u'feature', u'reduce', u'battery', u'consumption', u'dev', u'team', u'hat', u'do', u'not', u'get', u'notification', u'desk', u'slack', u'se

In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)], [(14, 1), (15, 1), (16, 1), (17, 1)], [(18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)], [(24, 1)], [(25, 1), (26, 1)], [(27, 1), (28, 1), (29, 1), (30, 1)], [(16, 1), (25, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1)], [(8, 4), (19, 2), (25, 2), (31, 2), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 2), (52, 1), (53, 1), (54, 1), (55, 2), (56, 3), (57, 1), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 1), (64, 2), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 3), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 2), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 5), (87, 2), (88, 1), (89, 1), (90, 1)], [(91, 1)], [(51, 1), (64, 1), (66, 1), (92, 1), (93, 1)], [(8, 1), (32, 1), (51, 1), (65, 1), (66, 1), (90, 1), (94, 1), 

In [15]:
id2word[1]

u'conversation'

In [16]:
# Human readable format of corpus (term-frequency)
#[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [17]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


In [18]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  u'0.031*"update" + 0.026*"new" + 0.025*"even" + 0.018*"make" + 0.015*"screen" + 0.013*"workspace" + 0.012*"app" + 0.012*"late" + 0.012*"bad" + 0.012*"much"'),
 (1,
  u'0.019*"able" + 0.018*"load" + 0.017*"easy" + 0.014*"amazing" + 0.013*"project" + 0.013*"back" + 0.013*"lot" + 0.011*"add" + 0.010*"galaxy" + 0.010*"launch"'),
 (2,
  u'0.077*"good" + 0.063*"great" + 0.036*"team" + 0.024*"mobile" + 0.019*"communication" + 0.019*"call" + 0.016*"slack" + 0.013*"slow" + 0.012*"nice" + 0.011*"sound"'),
 (3,
  u'0.045*"not" + 0.039*"work" + 0.034*"notification" + 0.024*"slack" + 0.023*"get" + 0.022*"time" + 0.022*"message" + 0.021*"do" + 0.016*"love" + 0.015*"can"')]


In [20]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      47.847523        1       1 -0.292514  0.056433
0      21.483759        1       2  0.153736  0.189416
2      19.110237        1       3  0.050329 -0.239574
1      11.558475        1       4  0.088449 -0.006275, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
928   Default  120.000000           good  120.000000  30.0000  30.0000
1297  Default   98.000000          great   98.000000  29.0000  29.0000
536   Default  178.000000            not  178.000000  28.0000  28.0000
392   Default  134.000000   notification  134.000000  27.0000  27.0000
423   Default   56.000000           team   56.000000  26.0000  26.0000
386   Default   54.000000         update   54.000000  25.0000  25.0000
67    Default  161.000000           work  161.000000  24.0000  24.0000
25    Default   46.000000            new   46.000000  23.0000  23.0000
23    Default   45.000000           even   45.000000  22.0000  22.0000
1015  Default   38.000000         mobile   38.000000  21.0000  21.0000
643   Default   89.000000            get   89.000000  20.0000  20.0000
1411  Default   86.000000           time   86.000000  19.0000  19.0000
202   Default   83.000000             do   83.000000  18.0000  18.0000
1419  Default  120.000000          slack  120.000000  17.0000  17.0000
760   Default   30.000000  communication   30.000000  16.0000  16.0000
272   Default   32.000000           make   32.000000  15.0000  15.0000
48    Default   30.000000           call   30.000000  14.0000  14.0000
664   Default   27.000000         screen   27.000000  13.0000  13.0000
1155  Default   62.000000           love   62.000000  12.0000  12.0000
1437  Default   18.000000           able   18.000000  11.0000  11.0000
1083  Default   17.000000           load   17.000000  10.0000  10.0000
127   Default   91.000000        message   91.000000   9.0000   9.0000
1290  Default   57.000000            can   57.000000   8.0000   8.0000
301   Default   16.000000           easy   16.000000   7.0000   7.0000
1452  Default   23.000000      workspace   23.000000   6.0000   6.0000
636   Default   21.000000           slow   21.000000   5.0000   5.0000
1306  Default   53.000000            use   53.000000   4.0000   4.0000
1302  Default   22.000000            app   22.000000   3.0000   3.0000
924   Default   22.000000           late   22.000000   2.0000   2.0000
211   Default   21.000000            bad   21.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
172    Topic4   12.593369        project   13.259560   2.1062  -4.3211
1183   Topic4   12.455835           back   13.214573   2.0986  -4.3321
1574   Topic4   10.789172            add   11.449049   2.0984  -4.4757
770    Topic4   11.916819            lot   12.656737   2.0975  -4.3763
1359   Topic4    9.905675         galaxy   10.580110   2.0919  -4.5612
966    Topic4    9.605696         launch   10.327220   2.0853  -4.5919
1074   Topic4    8.511800      interface    9.167233   2.0836  -4.7128
1195   Topic4    8.578451            run    9.251181   2.0823  -4.7050
599    Topic4    8.373112        browser    9.035227   2.0816  -4.7292
750    Topic4    7.473621    communicate    8.139142   2.0724  -4.8429
1389   Topic4    8.091232          cause    8.849946   2.0681  -4.7635
351    Topic4    7.508732     constantly    8.216681   2.0677  -4.8382
1119   Topic4    6.874700          error    7.523705   2.0675  -4.9264
1121   Topic4    6.937883           loop    7.625134   2.0633  -4.9173
1537   Topic4    5.939894         almost    6.586760   2.0544  -5.0726
279    Topic4    6.332325           hang    7.042599   2.0514  -5.0086
1360   Topic4    5.852975        perfect    6.511278   2.0512  -5.0873
540    Topic4    5.700794           drop    6.343102   2.0510  -5.1137
1434   Topic

In [18]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


('\nPerplexity: ', -6.413305381851651)
('\nCoherence Score: ', 0.4960519494995757)
